<a href="https://colab.research.google.com/github/teddycars/APPLE-ml-ferret/blob/main/Machine_Learning_Project_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we have to install all the libraries we are going to use for this project i.e. upgini, catboost and pandas.


In [22]:
%pip install -Uq upgini catboost

Prepare Data Input

Download the dataset from the link provided in the document.
Once the dataset has been downloaded, we are going to split that into a training dataset for training purposes and a testing dataset to test how accurate the model is.

In [23]:
from os.path import exists
import pandas as pd

df_path = "train.cvs.zip" if exists("train.cvs.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [24]:
train = df[df["date"]<"2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [25]:
train_features =train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features

In [26]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=dbefb885-79b7-43af-8edf-ce95b3fd90b5
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


68 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_34_c3ef5b4f,0.0497,100.0000,"5.5326, 5.1883, 5.469",Upgini,Weather & climate normals data,Daily
f_autofe_div_029f75a6,0.0175,100.0000,"0.8045, -0.634, 0.6102",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_pca_0_d7e0a1fc,0.0122,100.0000,"23.1658, -5.1359, -27.1183",Upgini,Weather & climate normals data,Daily
f_autofe_div_50507976,0.0102,100.0000,"0.0273, -0.0391, -0.0625",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_6793ee4f,0.0077,100.0000,"6.9487, 7.1495, -1.6416",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.0064,100.0000,"-11.5237, -18.3202, 6.5368",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_1edd5009,0.0061,100.0000,"0.0754, 0.0211, 0.0242",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_50a5dcaf,0.0058,100.0000,"-1.4562, -0.7524, 0.5427",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_6234ba3a,0.0055,100.0000,"0.5647, 0.7388, 1.0293",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_3e22df83,0.0042,100.0000,"0.01, -0.0079, 0.017",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_89c56a5f,0.0041,100.0000,"0.2202, -0.5213, 0.0947",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1136,45
Upgini,Weather & climate normals data,0.0674,5
Upgini,AutoFE: features from Markets data,0.0120,11
Upgini,Calendar data,0.0034,2
Upgini,AutoFE: features from Calendar data,0.0008,3
Upgini,Markets data,0.0007,1
Upgini,World economic indicators,0.0002,1


"Calendar data,Markets data",f_autofe_div_029f75a6,f_events_date_week_sin1_847b5db1,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,/
"Calendar data,Markets data",f_autofe_div_50507976,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,/
"Calendar data,Markets data",f_autofe_mul_6793ee4f,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,*
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_1edd5009,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,/
"Calendar data,Markets data",f_autofe_div_50a5dcaf,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,/
Markets data,f_autofe_mul_6234ba3a,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,*
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
"Calendar data,Markets data",f_autofe_div_534d01d5,f_events_date_week_sin1_847b5db1,f_financial_date_stoxx_043cbcd4,/


We detected 113 outliers in your sample.
Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,301.2609,193.9221,107.3388
Eval 1,3787,59.2424,485.3569,362.1067,123.2502


In [27]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.252850,0.161732,0.091118
1,Eval 1,3787,59.2424,0.264187,0.199200,0.064987


In [28]:
# Split the train_features DataFrame into two chunks
train_features_chunk1 = train_features[:7607]
train_features_chunk2 = train_features[7607:]

# Enrich each chunk separately
enriched_train_features_chunk1 = enricher.transform(train_features_chunk1, keep_input=True)
enriched_train_features_chunk2 = enricher.transform(train_features_chunk2, keep_input=True)

# Combine the enriched chunks
enriched_train_features = pd.concat([enriched_train_features_chunk1, enriched_train_features_chunk2])

# Enrich the test features
enriched_test_features = enricher.transform(test_features, keep_input=True)

# Display the head of the enriched train features
enriched_train_features.head()

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=0fd02294-2ba7-4f2f-9359-f543f2cfa6e0
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

...

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=29dc7988-6e36-4625-ac02-b6bb2e59b155
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_weather_date_weather_umap_34_c3ef5b4f,f_autofe_div_029f75a6,f_weather_date_weather_pca_0_d7e0a1fc,f_autofe_div_50507976,f_autofe_mul_6793ee4f,f_autofe_mul_34d11bc4,f_autofe_div_1edd5009,...,f_autofe_div_fb99ff57,f_autofe_mul_c883f8a6,f_events_date_year_sin2_59955ffd,f_autofe_mul_6a97c336,f_autofe_div_0016ede5,f_autofe_mul_44503c6f,f_autofe_mul_8c2f1d42,f_autofe_mul_8437d645,f_autofe_mul_3e749d53,f_autofe_div_ef42018c
0,2013-01-01,7,5,5.664261,0.683543,29.676683,0.032652,17.753667,29.727047,0.054674,...,280.574579,318.866728,0.337523,0.716951,100.766473,3.751472,-0.887653,49.735779,-1.030522,1.022516
1,2013-01-01,4,9,5.664261,0.683543,29.676683,0.032652,17.753667,29.727047,0.054674,...,280.574579,318.866728,0.337523,0.716951,100.766473,3.751472,-0.887653,49.735779,-1.030522,1.022516
2,2013-01-01,1,33,5.664261,0.683543,29.676683,0.032652,17.753667,29.727047,0.054674,...,280.574579,318.866728,0.337523,0.716951,100.766473,3.751472,-0.887653,49.735779,-1.030522,1.022516
3,2013-01-01,3,41,5.664261,0.683543,29.676683,0.032652,17.753667,29.727047,0.054674,...,280.574579,318.866728,0.337523,0.716951,100.766473,3.751472,-0.887653,49.735779,-1.030522,1.022516
4,2013-01-01,5,24,5.664261,0.683543,29.676683,0.032652,17.753667,29.727047,0.054674,...,280.574579,318.866728,0.337523,0.716951,100.766473,3.751472,-0.887653,49.735779,-1.030522,1.022516


In [29]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]

In [30]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[16.447734782782064]